<i>Copyright (c) Microsoft Corporation.</i>

<i>Licensed under the MIT License.</i>

# Arima: Autoregressive Integrated Moving Average

This notebook provides an example of how to train an Arima model to generate point forecasts of product sales in retail. We will train an ARIMA based model on the Orange Juice dataset.

An ARIMA, which stands for **A**uto**r**egressive **I**ntegrated **M**oving **A**verage, model can be created using the `statsmodels` library. In the next section, we perform the following steps:

1. Define the model by calling `SARIMAX()` and passing in the model parameters: `p`, `d`, and `q` parameters, and `P`, `D`, and `Q` parameters

2. The model is prepared on the training data by calling the `fit()` function.

3. Predictions can be made by calling the `forecast()` function and specifying the number of steps (horizon) which to forecast. In an ARIMA model there are 3 parameters that are used to help model the major aspects of a times series: seasonality, trend, and noise. These parameters are:
    - **p** is the parameter associated with the auto-regressive aspect of the model, which incorporates past values.
    - **d** is the parameter associated with the integrated part of the model, which effects the amount of differencing to apply to a time series.
    - **q** is the parameter associated with the moving average part of the model.,

If our model has a seasonal component, we use a seasonal ARIMA model (SARIMA). In that case we have another set of parameters: `P`, `D`, and `Q` which describe the same associations as `p`, `d`, and `q`, but correspond with the seasonal components of the model.


## Global Settings and Imports

In [1]:
import os
import sys
import math
import warnings
import numpy as np
import pandas as pd
import scrapbook as sb

from pyramid.arima import auto_arima

from fclib.common.utils import git_repo_path
from fclib.evaluation.evaluation_utils import MAPE
from fclib.dataset.ojdata import download_ojdata, split_train_test
from fclib.feature_engineering.feature_utils import df_from_cartesian_product

warnings.filterwarnings("ignore")

print("System version: {}".format(sys.version))

System version: 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]
LightGBM version: 2.3.0


## Parameters


In what follows, we define global settings related to the model and feature engineering. LightGBM supports both classification models and regression models. In our case, we set the objective function to `mape` which stands for mean absolute percentage error (MAPE) since we will build a regression model to predict product sales and evaluate the accuracy of the model using MAPE.

Generally, we can adjust the number of leaves (`num_leaves`), the minimum number of data in each leaf (`min_data_in_leaf`), maximum number of boosting rounds (`num_rounds`), the learning rate of trees (`learning_rate`) and `early_stopping_rounds` (to avoid overfitting) in the model to get better performance. Besides, we can also adjust other supported parameters to optimize the results. [In this link](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.rst), a list of all the parameters is given. In addition, advice on how to tune these parameters can be found [in this url](https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters-Tuning.rst).

We will use historical weekly sales, date time information, and product information as input features to train the model. Prior sales are used as lag features and `lags` contains the lags where each number indicates the number of time steps (i.e., weeks) that we shift the data backwards to get the historical sales. We also use the average sales within a certain time window in the past as a moving average feature. `window_size` controls the size of the moving window. Apart from these parameters, we use `use_columns` and `categ_fea` to denote all other features that we leverage in the model and the categorical features, respectively.


In [2]:
# Use False if you've already downloaded and split the data
DOWNLOAD_SPLIT_DATA = True

# Data directory
DATA_DIR = os.path.join(git_repo_path(), "ojdata")

# Forecasting settings
N_SPLITS = 1
HORIZON = 2
GAP = 2
FIRST_WEEK = 40
LAST_WEEK = 138

# Parameters of LightGBM model
params = {
    "objective": "mape",
    "num_leaves": 124,
    "min_data_in_leaf": 340,
    "learning_rate": 0.1,
    "feature_fraction": 0.65,
    "bagging_fraction": 0.87,
    "bagging_freq": 19,
    "num_rounds": 940,
    "early_stopping_rounds": 125,
    "num_threads": 16,
    "seed": 1,
}

# Lags, window size, and feature column names
lags = np.arange(2, 20)
window_size = 40
used_columns = ["store", "brand", "week", "week_of_month", "month", "deal", "feat", "move", "price", "price_ratio"]
categ_fea = ["store", "brand", "deal"]

## Data Preparation

We need to download the Orange Juice data and split it into training and test sets. By default, the following cell will download and spit the data. If you've already done so, you may skip this part by switching `DOWNLOAD_SPLIT_DATA` to `False`.

We store the training data and test data using dataframes. The training data includes `train_df` and `aux_df` with `train_df` containing the historical sales up to week 135 (the time we make forecasts) and `aux_df` containing price/promotion information up until week 138. Here we assume that future price and promotion information up to a certain number of weeks ahead is predetermined and known. The test data is stored in `test_df` which contains the sales of each product in week 137 and 138. Assuming the current week is week 135, our goal is to forecast the sales in week 137 and 138 using the training data. There is a one-week gap between the current week and the first target week of forecasting as we want to leave time for planning inventory in practice.

The setting of the forecast problem are defined in `fclib.dataset.ojdata.split_train_test` function. We can change this setting (e.g., modify the horizon of the forecast or the range of the historical data) by passing different parameters to this functions. Below, we split the data into `n_splits=1` splits, using the forecasting settings listed above in the **Parameters** section.

In [112]:
if DOWNLOAD_SPLIT_DATA:
    download_ojdata(DATA_DIR)
    train_df_list, test_df_list, aux_df_list = split_train_test(
        DATA_DIR,
        n_splits=N_SPLITS,
        horizon=HORIZON,
        gap=GAP,
        first_week=FIRST_WEEK,
        last_week=LAST_WEEK,
        write_csv=True,
    )

    # Split returns a list, extract the dataframes from the list
    train_df = train_df_list[0].reset_index()
    test_df = test_df_list[0].reset_index()
    aux_df = aux_df_list[0].reset_index()

Data already exists at the specified location.


Our data preparation for the training set will involve the following steps:

- Filter the original dataset to include only that time period reserved for the training set
- Scale the time series such that the values fall within the interval (0, 1)

Note that the training set only contains the model features.

In [4]:
# Select only required columns
train_df["move"] = train_df["logmove"].apply(lambda x: round(math.exp(x)))
train_df = train_df[["store", "brand", "week", "move"]]
train_df.head(10)

,store,brand,week,move
0,2,1,40,8256
1,2,1,46,6144
2,2,1,47,3840
3,2,1,48,8000
4,2,1,50,8896
5,2,1,51,7168
6,2,1,52,10880
7,2,1,53,7744
8,2,1,54,8512
9,2,1,57,5504


In [5]:
# Create a dataframe to hold all necessary data
store_list = train_df["store"].unique()
brand_list = train_df["brand"].unique()
week_list = range(FIRST_WEEK, LAST_WEEK + 1)
d = {"store": store_list, "brand": brand_list, "week": week_list}
data_grid = df_from_cartesian_product(d)
data_filled = pd.merge(data_grid, train_df, how="left", on=["store", "brand", "week"])
data_filled.head(10)

,store,brand,week,move
0,2,1,40,8256.0
1,2,1,41,NaN
2,2,1,42,NaN
3,2,1,43,NaN
4,2,1,44,NaN
5,2,1,45,NaN
6,2,1,46,6144.0
7,2,1,47,3840.0
8,2,1,48,8000.0
9,2,1,49,NaN


In [6]:
# Fill missing values
print("Number of missing rows is {}".format(data_filled[data_filled.isnull().any(axis=1)].shape[0]))
data_filled = data_filled.groupby(["store", "brand"]).apply(lambda x: x.fillna(method="ffill").fillna(method="bfill"))
data_filled.head(10)

Number of missing rows is 6204


,store,brand,week,move
0,2,1,40,8256.0
1,2,1,41,8256.0
2,2,1,42,8256.0
3,2,1,43,8256.0
4,2,1,44,8256.0
5,2,1,45,8256.0
6,2,1,46,6144.0
7,2,1,47,3840.0
8,2,1,48,8000.0
9,2,1,49,8000.0


## Model training

We next train an ARIMA model.

In [95]:
STORE = 132
BRAND = 2
train_ts = data_filled.loc[(data_filled.store == STORE) & (data_filled.brand == BRAND)]
train_ts.head(20)

,store,brand,week,move
87219,132,2,40,6624.0
87220,132,2,41,7008.0
87221,132,2,42,7008.0
87222,132,2,43,6144.0
87223,132,2,44,5856.0
87224,132,2,45,6432.0
87225,132,2,46,5952.0
87226,132,2,47,6432.0
87227,132,2,48,4128.0
87228,132,2,49,7104.0


In [96]:
train_ts = np.array(train_ts["move"])

In [132]:
model = auto_arima(train_ts, seasonal=False, start_p=0, start_q=0, max_p=10, max_q=10)  # seasonal=True, m=52)

In [133]:
model.fit(train_ts)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 1),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=None, solver='lbfgs', start_params=None,
   suppress_warnings=False, transparams=True, trend='c')

In [134]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             ARIMA Model Results                              
==============================================================================
Dep. Variable:                    D.y   No. Observations:                   98
Model:                 ARIMA(0, 1, 1)   Log Likelihood                -913.764
Method:                       css-mle   S.D. of innovations           2648.474
Date:                Tue, 25 Feb 2020   AIC                           1833.528
Time:                        13:58:00   BIC                           1841.283
Sample:                             1   HQIC                          1836.665
                                                                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         30.9574      9.314      3.324      0.001      12.701      49.213
ma.L1.D.y     -1.0000      0.034    -29.233      0.000      -1.067      -0.933
                                    Roots                                    
=============================================================================
                  Real          Imaginary           Modulus         Frequency
-----------------------------------------------------------------------------
MA.1            1.0000           +0.0000j            1.0000            0.0000
-----------------------------------------------------------------------------
"""

In [135]:
preds = model.predict(n_periods=GAP + HORIZON)
preds = preds[GAP:]
preds

array([9428.69132636, 9459.64871203])

## Model evaluation

In [136]:
# Evaluate prediction accuracy
test_df["actual"] = test_df["logmove"].apply(lambda x: round(math.exp(x)))

In [137]:
test_df.head()

,store,brand,week,logmove,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit,actual
0,2,1,137,9.189321,1,0.041645,0.051979,0.047656,0.038801,0.032656,0.038125,0.032861,0.036094,0.037344,0.022187,0.032422,0,0.0,20.425098,9792
1,2,1,138,9.738613,1,0.037344,0.038958,0.047656,0.035781,0.043594,0.050937,0.042031,0.038906,0.037344,0.031094,0.032422,1,1.0,11.290000,16960
2,2,2,137,8.738735,1,0.041645,0.051979,0.047656,0.038801,0.032656,0.038125,0.032861,0.036094,0.037344,0.022187,0.032422,0,0.0,33.300308,6240
3,2,2,138,9.601301,1,0.037344,0.038958,0.047656,0.035781,0.043594,0.050937,0.042031,0.038906,0.037344,0.031094,0.032422,1,1.0,9.430000,14784
4,2,3,137,7.560080,1,0.041645,0.051979,0.047656,0.038801,0.032656,0.038125,0.032861,0.036094,0.037344,0.022187,0.032422,0,0.0,30.506667,1920


In [138]:
store_list = train_df["store"].unique()
brand_list = train_df["brand"].unique()
week_list = range(LAST_WEEK - HORIZON + 1, LAST_WEEK + 1)
d = {"store": store_list, "brand": brand_list, "week": week_list}
data_grid = df_from_cartesian_product(d)
test_filled = pd.merge(data_grid, test_df, how="left", on=["store", "brand", "week"])
test_filled.head(10)

,store,brand,week,logmove,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit,actual
0,2,1,137,9.189321,1,0.041645,0.051979,0.047656,0.038801,0.032656,0.038125,0.032861,0.036094,0.037344,0.022187,0.032422,0,0.0,20.425098,9792
1,2,1,138,9.738613,1,0.037344,0.038958,0.047656,0.035781,0.043594,0.050937,0.042031,0.038906,0.037344,0.031094,0.032422,1,1.0,11.290000,16960
2,2,2,137,8.738735,1,0.041645,0.051979,0.047656,0.038801,0.032656,0.038125,0.032861,0.036094,0.037344,0.022187,0.032422,0,0.0,33.300308,6240
3,2,2,138,9.601301,1,0.037344,0.038958,0.047656,0.035781,0.043594,0.050937,0.042031,0.038906,0.037344,0.031094,0.032422,1,1.0,9.430000,14784
4,2,3,137,7.560080,1,0.041645,0.051979,0.047656,0.038801,0.032656,0.038125,0.032861,0.036094,0.037344,0.022187,0.032422,0,0.0,30.506667,1920
5,2,3,138,7.249926,1,0.037344,0.038958,0.047656,0.035781,0.043594,0.050937,0.042031,0.038906,0.037344,0.031094,0.032422,0,0.0,33.145455,1408
6,2,4,137,7.592870,1,0.041645,0.051979,0.047656,0.038801,0.032656,0.038125,0.032861,0.036094,0.037344,0.022187,0.032422,0,0.0,19.855806,1984
7,2,4,138,9.300547,1,0.037344,0.038958,0.047656,0.035781,0.043594,0.050937,0.042031,0.038906,0.037344,0.031094,0.032422,1,1.0,13.129474,10944
8,2,5,137,9.852615,1,0.041645,0.051979,0.047656,0.038801,0.032656,0.038125,0.032861,0.036094,0.037344,0.022187,0.032422,1,1.0,7.159817,19008
9,2,5,138,8.269757,1,0.037344,0.038958,0.047656,0.035781,0.043594,0.050937,0.042031,0.038906,0.037344,0.031094,0.032422,0,0.0,28.573143,3904


In [139]:
test = test_df.loc[(test_df.store == STORE) & (test_df.brand == BRAND)]
test.head(20)

,store,brand,week,logmove,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit,actual
1762,132,2,137,8.995165,1,0.040369,0.043646,0.041406,0.037038,0.027969,0.031146,0.031937,0.030156,0.027969,0.020781,0.025703,0,0.0,20.529048,8064
1763,132,2,138,9.920934,1,0.033750,0.032708,0.041406,0.031406,0.037344,0.041563,0.035781,0.031094,0.027969,0.026406,0.025703,1,1.0,7.787406,20352


In [140]:
metric_value = MAPE(test["actual"], preds) * 100
sb.glue("MAPE", metric_value)
print("MAPE of the forecasts is {}".format(metric_value))

MAPE of the forecasts is 64.80961239832934
